In [1]:
import re
import pandas as pd
from sklearn.pipeline import Pipeline
import tensorflow as tf

In [2]:
mix = pd.read_csv('test.csv')

In [3]:
mix.columns

Index(['Unnamed: 0', 'film-url', 'review', 'polarity'], dtype='object')

In [4]:
mix.drop(columns=['Unnamed: 0','film-url'], inplace=True)

In [5]:
print(mix.head(5))
print(len(mix))

                                              review  polarity
0  Magnifique épopée, une belle histoire, touchan...         1
1  Je n'ai pas aimé mais pourtant je lui mets 2 é...         0
2  Un dessin animé qui brille par sa féerie et se...         1
3  Si c'est là le renouveau du cinéma français, c...         0
4  Et pourtant on s’en Doutait !Second volet très...         0
20000


In [6]:
pos_review =[]
neg_review = []
for i in range(len(mix)):
  if mix['polarity'][i] == 1:
    pos_review.append(mix['review'][i])
for i in range(len(mix)):
  if mix['polarity'][i] == 0:
    neg_review.append(mix['review'][i])
pos = pd.DataFrame(data=pos_review,columns=['review'])
neg = pd.DataFrame(data=neg_review,columns=['review'])

In [7]:
print(pos.head(5))
print(neg.head(5))

                                              review
0  Magnifique épopée, une belle histoire, touchan...
1  Un dessin animé qui brille par sa féerie et se...
2  Vous reprendrez bien un peu d'été ? Ce film je...
3  Bon c'est pas un grand film mais on passe un b...
4  Terrible histoire que ces êtres sans amour, ce...
                                              review
0  Je n'ai pas aimé mais pourtant je lui mets 2 é...
1  Si c'est là le renouveau du cinéma français, c...
2  Et pourtant on s’en Doutait !Second volet très...
3  Mais comment certaines personnes ont pus lui m...
4  On ne comprend que guère l'espace et le temps ...


In [8]:
len(neg)

10408

In [9]:
pos_train=[]
pos_test =[]
neg_train=[]
neg_test =[]
for i in range(len(pos)):
  if i< 5000:
    pos_train.append(pos['review'][i])
  if i in range(5000,6250):
    pos_test.append(pos['review'][i])
for i in range(len(neg)):
  if i< 5000:
    neg_train.append(neg['review'][i])
  if i in range(5000,6250):
    neg_test.append(neg['review'][i])
train_pos = pd.DataFrame(data=pos_train,columns=['commentaire'])
train_neg = pd.DataFrame(data=neg_train,columns=['commentaire'])
test_pos = pd.DataFrame(data=pos_test,columns=['commentaire'])
test_neg = pd.DataFrame(data=neg_test,columns=['commentaire'])

In [10]:
test_neg.shape
#pos_train.head(5)

(1250, 1)

In [11]:
test_pos['label'] = 1
for i in range(len(test_pos)):
   file = open ('allocine/test/pos/file_pos'+str(i)+'.txt','w')
   file.write(str(test_pos['commentaire'][i]))
   file.close()

In [12]:
test_neg['label'] = 0
for i in range(len(test_neg)):
   file = open ('allocine/test/neg/file_neg'+str(i)+'.txt','w')
   file.write(str(test_neg['commentaire'][i]))
   file.close()

In [13]:
train_pos['label'] = 1
for i in range(len(train_pos)):
   file = open ('allocine/train/pos/file_pos'+str(i)+'.txt','w')
   file.write(str(train_pos['commentaire'][i]))
   file.close()

In [14]:
train_neg['label'] = 0
for i in range(len(train_neg)):
   file = open ('allocine/train/neg/file_neg'+str(i)+'.txt','w')
   file.write(str(train_neg['commentaire'][i]))
   file.close()

In [15]:
!pip install transformers

     |████████████████████████████████| 2.1MB 7.7MB/s 
     |████████████████████████████████| 3.3MB 39.2MB/s 
     |████████████████████████████████| 901kB 40.1MB/s 


In [16]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

In [17]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
import tensorflow as tf

In [19]:
train = tf.keras.preprocessing.text_dataset_from_directory(
    'allocine/train', batch_size=15000, validation_split=0.2, 
    subset='training', seed=123)
test = tf.keras.preprocessing.text_dataset_from_directory(
    'allocine/train', batch_size=15000, validation_split=0.2, 
    subset='validation', seed=123)

Found 10000 files belonging to 3 classes.
Using 8000 files for training.
Found 10000 files belonging to 3 classes.
Using 2000 files for validation.


In [20]:
for i in train.take(1):
    train_feat = i[0].numpy()
    train_lab = i[1].numpy()

train = pd.DataFrame([train_feat, train_lab]).T
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
train['DATA_COLUMN'] = train['DATA_COLUMN'].str.decode("utf-8")
train.loc[train['LABEL_COLUMN'] == 1, ['LABEL_COLUMN']] = 0
train.loc[train['LABEL_COLUMN'] == 2, ['LABEL_COLUMN']] = 1
train.head()

,DATA_COLUMN,LABEL_COLUMN
0,"Aucun spolier dans ma critique, juste un synop...",1
1,J'ai trouvé ce film assez long... loin sont le...,0
2,"Nullissime!! Aucun scénario, clichés sur clich...",0
3,Un film assez saisissant d'avant-gardisme dans...,1
4,"Très moyen, voire mauvais, un téléfilm qui n’e...",0


In [21]:
for j in test.take(1):
    test_feat = j[0].numpy()
    test_lab = j[1].numpy()

test = pd.DataFrame([test_feat, test_lab]).T
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test['DATA_COLUMN'] = test['DATA_COLUMN'].str.decode("utf-8")
test.loc[test['LABEL_COLUMN'] == 1, ['LABEL_COLUMN']] = 0
test.loc[test['LABEL_COLUMN'] == 2, ['LABEL_COLUMN']] = 1
test.head()

,DATA_COLUMN,LABEL_COLUMN
0,Encore une critique qui n'est pas passée... Je...,0
1,1946. Dans un petit village d’Italie. Un curé ...,0
2,Les acteurs sont exellent et les effets spécia...,1
3,Je viens de le revoir pour la cinquième fois e...,1
4,C'est une toute petite aventure. La chasse du ...,0


In [22]:
!nvidia-smi

Tue May  4 22:55:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    35W / 250W |   1441MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [23]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .
import os
os.chdir("bert-sklearn")
print(os.listdir())

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 10.53 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Processing /content/bert-sklearn
     |████████████████████████████████| 133kB 9.1MB/s 
     |████████████████████████████████| 81kB 5.4MB/s 
     |████████████████████████████████| 7.5MB 13.2MB/s 
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-cp37-none-any.whl size=54235 sha256=c9c43ebd9b83e0a1be00ab436bb80d2eb9fb9078a85a3aae07846994323d367e
  Stored in directory: /root/.cache/pip/wheels/61/95/c6/5790aae8fb377f5ff356dbe58205aab28858595d6bff8197d0
Successfully built bert-sklearn
ERROR: botocore 1.20.65 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
['README.md', '.git', 'demo.ipynb', 'other_examples', 'demo_tuning_hyperparams.ipynb', 'tests', 'Options.md', 'glue_examples', 'setup.py', '

In [24]:
import torch
print('pytorch version:', torch.__version__)
print('GPU:',torch.cuda.get_device_name(0))

pytorch version: 1.8.1+cu101
GPU: Tesla P100-PCIE-16GB


In [25]:
from bert_sklearn import BertClassifier

In [ ]:
for i in range(0,len(train)):
  if train['LABEL_COLUMN'][i]==1:
    train.loc[i,'LABEL_COLUMN']='positif'
  else :
    train.loc[i,'LABEL_COLUMN']='negatif'
    

In [ ]:
from sklearn.model_selection import GridSearchCV


params = {'epochs':[2, 3, 4], 'learning_rate':[2e-5, 3e-5, 5e-5]}

# wrap classifier/regressor in GridSearchCV
clf = GridSearchCV(BertClassifier(validation_fraction=0, max_seq_length=64), 
                   params,
                   cv=3,
                   scoring='accuracy',
                   verbose=True)

# fit gridsearch 
clf.fit(train['DATA_COLUMN'] ,train['LABEL_COLUMN'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
100%|██████████| 231508/231508 [00:00<00:00, 20706592.11B/s]

Building sklearn text classifier...
Building sklearn text classifier...
Fitting 3 folds for each of 9 candidates, totalling 27 fits
Building sklearn text classifier...
Loading bert-base-uncased model...



100%|██████████| 433/433 [00:00<00:00, 139327.47B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 5333, validation data size: 0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


...


Building sklearn text classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 5334, validation data size: 0




Building sklearn text classifier...
Building sklearn text classifier...


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 120.5min finished


Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 8000, validation data size: 0



GridSearchCV(cv=3, error_score=nan,
             estimator=BertClassifier(bert_config_json=None,
                                      bert_model='bert-base-uncased',
                                      bert_vocab=None, do_lower_case=None,
                                      epochs=3, eval_batch_size=8, fp16=False,
                                      from_tf=False,
                                      gradient_accumulation_steps=1,
                                      ignore_label=None, label_list=None,
                                      learning_rate=2e-05, local_rank=-1,
                                      logfile='bert_sklearn.log', loss_scale=0,
                                      max_seq_length=64, num_mlp_hiddens=500,
                                      num_mlp_layers=0, random_state=42,
                                      restore_file=None, train_batch_size=32,
                                      use_cuda=True, validation_fraction=0,
                        

In [ ]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        
# best scores
print("\nBest score:", clf.best_score_,"with params:", clf.best_params_)

0.777 (+/-0.053) for {'epochs': 2, 'learning_rate': 2e-05}
0.790 (+/-0.027) for {'epochs': 2, 'learning_rate': 3e-05}
0.686 (+/-0.261) for {'epochs': 2, 'learning_rate': 5e-05}
0.787 (+/-0.027) for {'epochs': 3, 'learning_rate': 2e-05}
0.799 (+/-0.016) for {'epochs': 3, 'learning_rate': 3e-05}
0.680 (+/-0.256) for {'epochs': 3, 'learning_rate': 5e-05}
0.803 (+/-0.009) for {'epochs': 4, 'learning_rate': 2e-05}
0.795 (+/-0.026) for {'epochs': 4, 'learning_rate': 3e-05}
0.582 (+/-0.237) for {'epochs': 4, 'learning_rate': 5e-05}

Best score: 0.8030008064445808 with params: {'epochs': 4, 'learning_rate': 2e-05}


In [26]:
InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1)

In [29]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
    train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

    validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

    return train_InputExamples, validation_InputExamples

    train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [30]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-05, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [ ]:
for i in range(0,len(train)):
  if train['LABEL_COLUMN'][i]=='positif':
    train.loc[i,'LABEL_COLUMN']=1
  else :
    train.loc[i,'LABEL_COLUMN']=0

In [32]:
model.fit(train_data, epochs=4, validation_data=validation_data)

Epoch 1/4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


05/04/2021 22:58:09 - WARNING - tensorflow -   AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x7f7df1cfed00>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


05/04/2021 22:58:08 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
05/04/2021 22:58:10 - WARNING - tensorflow -   AutoGraph could not transform <function wrap at 0x7f7e0d5aad40> and will run it as-is.
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

05/04/2021 22:59:02 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


    500/Unknown - 316s 492ms/step - loss: 0.6080 - accuracy: 0.6333WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


05/04/2021 23:03:22 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


05/04/2021 23:03:22 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


500/500 [==============================] - 331s 521ms/step - loss: 0.6078 - accuracy: 0.6335 - val_loss: 0.3358 - val_accuracy: 0.8490
Epoch 2/4
500/500 [==============================] - 257s 514ms/step - loss: 0.2320 - accuracy: 0.9109 - val_loss: 0.4432 - val_accuracy: 0.8555
Epoch 3/4
500/500 [==============================] - 257s 513ms/step - loss: 0.0902 - accuracy: 0.9668 - val_loss: 0.5937 - val_accuracy: 0.8560
Epoch 4/4
500/500 [==============================] - 257s 514ms/step - loss: 0.0352 - accuracy: 0.9877 - val_loss: 0.7357 - val_accuracy: 0.8475


In [1]:
pred_sentences = ['orange est le plus mauvais operateur']
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
    print(pred_sentences[i], ": \n", labels[label[i]])

NameError: name 'tokenizer' is not defined

In [ ]:
pred_sentences = ['orange est le plus meilleur operateur']
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
    print(pred_sentences[i], ": \n", labels[label[i]])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report,precision_score,accuracy_score,f1_score
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
trueSTR = test.copy()
trueSTR.loc[test['LABEL_COLUMN'] == 0, 'LABEL_COLUMN'] = 'negative'
trueSTR.loc[test['LABEL_COLUMN'] == 1, 'LABEL_COLUMN'] = 'positive'

In [ ]:
predictedSTR =  []
for i in range(len(test)):
    pred_sentences = str(test.at[i,'DATA_COLUMN'])
    tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
    tf_outputs = model(tf_batch)
    tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
    labels = [0,1]
    label = tf.argmax(tf_predictions, axis=1)
    label = label.numpy()
    if labels[label[0]] == 0:
        predictedSTR.append('negative')
    else:
        predictedSTR.append('positive')

In [ ]:
predictedINT =  []
for i in range(len(test)):
    pred_sentences = str(test.at[i,'DATA_COLUMN'])
    tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
    tf_outputs = model(tf_batch)
    tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
    labels = [0,1]
    label = tf.argmax(tf_predictions, axis=1)
    label = label.numpy()
    predictedINT.append(labels[label[0]]) 

In [ ]:
class_names = ["positive", "negative"]
def show_confusion_matrix(confusion_matrix):
    hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
    hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
    hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
    plt.ylabel('True sentiment')
    plt.xlabel('Predicted sentiment');
cm = confusion_matrix(trueSTR['LABEL_COLUMN'], predictedSTR)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)

In [ ]:
print(classification_report(trueSTR['LABEL_COLUMN'], predictedSTR, target_names = class_names))

In [27]:
len(train)

8000

In [28]:
len(test)

2000